# Weighted Interval Scheduling on Medical Appointment Dataset

In [35]:
# Import files
import sys
from pathlib import Path
import os
import pandas as pd
import python.weighed_interval_scheduling as wis

sys.path.append(str(Path.cwd().parent))

days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
medical_path = "../data/processed-datasets/medical-appointment-scheduling-system/medical-appointments-"
cloud_path = "../data/processed-datasets/cloud-workload-job-traces/cloud-workload-"

In [36]:
# Import medical data
dfs = []
for day in days:
    filepath = medical_path + day + ".csv"
    if Path(filepath).exists():
        df = pd.read_csv(filepath)
    dfs.append(df)

In [37]:
# Data frame columns
interval_col = 'appointment_id'
start_time_col = 'start_time'
end_time_col = 'end_time'
start_min_col = 'start_minutes'
end_min_col = 'end_minutes'
priority_col = 'priority'

In [38]:
# Run wis algorithm on datasets
for df in dfs:
    # Convert dataframes to lists
    interval = df[interval_col].to_list()
    intervalNum = [i for i in range(len(interval))]
    start = df[start_min_col].to_list()
    finish = df[end_min_col].to_list()
    weight = df[priority_col].to_list()

    # Run algorithm
    max_profit, selected_intervals = wis.create_interval_schedule(intervalNum, start, finish, weight)
    selected = [interval[i] for i in selected_intervals]
    print(f"Maximum Profit: {max_profit}")
    print(f"Selected Jobs: {selected}")
    print(f"Number of jobs selected: {len(selected)}")
    print(f"Number of jobs not selected: {len(df) - len(selected)}\n")     


Maximum Profit: 2806
Selected Jobs: [564, 102, 309, 182, 2, 543, 354, 454, 782, 327, 793, 478, 386, 174, 627, 714, 293, 616, 481, 490, 139, 678, 19, 657, 428, 546, 85, 520, 95, 8, 321, 266, 503, 776, 76, 119, 391, 746, 167, 3, 767, 74, 589, 284, 96, 316, 287]
Number of jobs selected: 47
Number of jobs not selected: 56

Maximum Profit: 2694
Selected Jobs: [479, 171, 717, 716, 337, 70, 773, 285, 317, 161, 683, 574, 43, 788, 353, 580, 772, 28, 384, 703, 507, 346, 595, 10, 688, 659, 493, 608, 121, 268, 370, 604, 219, 800, 730, 281, 52, 486, 156, 679, 576, 799, 456, 586, 527, 553, 692, 690]
Number of jobs selected: 48
Number of jobs not selected: 50

Maximum Profit: 2924
Selected Jobs: [395, 821, 494, 301, 536, 572, 530, 185, 635, 859, 324, 645, 137, 38, 62, 361, 639, 528, 372, 412, 385, 300, 256, 614, 409, 159, 423, 98, 607, 226, 681, 231, 596, 65, 648, 404, 209, 656, 765, 378, 302, 487, 305, 721, 27, 820, 200]
Number of jobs selected: 47
Number of jobs not selected: 58

Maximum Profit: 28

# Weighted Interval Scheduling on Cloud Workload Datasset

In [39]:
# Import cloud data
dfs = []
for day in days:
    filepath = cloud_path + day + ".csv"
    if Path(filepath).exists():
        df = pd.read_csv(filepath)
    dfs.append(df)

In [40]:
# Data frame columns
interval_col = 'job_interval'
start_time_col = 'start_time'
end_time_col = 'end_time'
start_min_col = 'start_minutes'
end_min_col = 'end_minutes'
priority_col = 'priority'

In [41]:
# Run wis algorithm on datasets
for df in dfs:
    # Convert dataframes to lists
    interval = df[interval_col].to_list()
    intervalNum = [i for i in range(len(interval))]
    start = df[start_min_col].to_list()
    finish = df[end_min_col].to_list()
    weight = df[priority_col].to_list()

    # Run algorithm
    max_profit, selected_intervals = wis.create_interval_schedule(intervalNum, start, finish, weight)
    selected = [interval[i] for i in selected_intervals]
    print(f"Maximum Profit: {max_profit}")
    print(f"Selected Jobs: {selected}")
    print(f"Number of jobs selected: {len(selected)}")
    print(f"Number of jobs not selected: {len(df) - len(selected)}\n")     


Maximum Profit: 53
Selected Jobs: ['JOB_00197', 'JOB_01164', 'JOB_02169', 'JOB_00560', 'JOB_00328', 'JOB_01232', 'JOB_01056', 'JOB_00176', 'JOB_00454', 'JOB_02133', 'JOB_00656', 'JOB_00734', 'JOB_01732', 'JOB_03233', 'JOB_03255', 'JOB_02863', 'JOB_00252', 'JOB_01796', 'JOB_00422', 'JOB_02562', 'JOB_03241', 'JOB_00398', 'JOB_01358', 'JOB_03307', 'JOB_00466', 'JOB_00910']
Number of jobs selected: 26
Number of jobs not selected: 11

Maximum Profit: 57
Selected Jobs: ['JOB_02875', 'JOB_00161', 'JOB_03259', 'JOB_03433', 'JOB_00859', 'JOB_01459', 'JOB_02839', 'JOB_00981', 'JOB_01574', 'JOB_01280', 'JOB_03335', 'JOB_03189', 'JOB_02160', 'JOB_02635', 'JOB_03266', 'JOB_00927', 'JOB_02017', 'JOB_01824', 'JOB_02672', 'JOB_02135', 'JOB_00791', 'JOB_00730', 'JOB_00805', 'JOB_01838', 'JOB_00246', 'JOB_02962', 'JOB_01387']
Number of jobs selected: 27
Number of jobs not selected: 13

Maximum Profit: 66
Selected Jobs: ['JOB_03010', 'JOB_02543', 'JOB_00521', 'JOB_00636', 'JOB_00499', 'JOB_01477', 'JOB_0